In [ ]:
!pip install MetaTrader5
!pip install pandas_datareader

In [1]:
import MetaTrader5 as mt5
from datetime import datetime
import time
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas_datareader import data as pdr
from datetime import date
import os
import warnings
import sys, time
from IPython.display import display, clear_output
warnings.filterwarnings("ignore")

In [ ]:
mt5.shutdown()

In [3]:
# estabelecemos a conexão ao MetaTrader 5
# conecte-se ao MetaTrader 5
mt5.initialize()
mt5.login(login=********,password="******")
# mt5.shutdown()

True

In [4]:
def get_ohlc(ativo, timeframe, n=20, periodo = 13, desvios = 2, window = 8):
    ativo = mt5.copy_rates_from_pos(ativo,timeframe, 0, n)
    ativo = pd.DataFrame(ativo)
    ativo['time'] = pd.to_datetime(ativo['time'], unit='s')
    ativo=ativo.reset_index()
    df=ativo
    df['Datetime'] = df['time']
    df['Data'] = [d.date() for d in df['Datetime']]
    df['Hora_Minuto'] = [d.time() for d in df['Datetime']]

    #calculo das bandas de Bollinger
    df['desvio'] = df['close'].rolling(periodo).std()
    df['MM'] = df['close'].rolling(periodo).mean()
    df['Banda_Sup'] = df['MM'] + (df['desvio']*desvios)
    df['Banda_Inf'] = df['MM'] - (df['desvio']*desvios) 
    
    #Calculo do indicador RSI
    #1º) Calcular Variação
    df['change'] = df['close'] - df['close'].shift(1)
    #2º) Identificar/Seprara entre periodos de ganho e perda
    df['gain'] = df.loc[df['change']>0,'change'].apply(abs)
    df.loc[(df['gain'].isna()),'gain'] = 0
    #df.loc[0,'gain'] = np.NaN
    df['loss'] = df.loc[df['change']<0,'change'].apply(abs)
    df.loc[(df['loss'].isna()),'loss'] = 0
    #df.loc[0,'loss'] = np.NaN
    #3º) Calcular Media Movel (Ganhos e Perdas)
    #window = 6
    df['avg_gain'] = df['gain'].rolling(window).mean()
    df['avg_loss'] = df['loss'].rolling(window).mean()
    first = df['avg_gain'].first_valid_index()
    #print(first)
    for index,row in df.iterrows():
        if index == first:
            prev_avg_gain = row['avg_gain']
            prev_avg_loss = row['avg_loss']
        elif index > first:
            df.loc[index,'avg_gain'] = ((prev_avg_gain*(window-1)) + row['gain'])/window
            prev_avg_gain = df.loc[index,'avg_gain']
            df.loc[index,'avg_loss'] = ((prev_avg_loss*(window-1)) + row['loss'])/window
            prev_avg_loss = df.loc[index,'avg_loss']
    #4º) Calcular RS
    df['RS14'] = df['avg_gain']/df['avg_loss']
    #5º) Calcular RSI
    df['RSI14'] = 100 - (100/(1 + df['RS14']))
    #df.tail()
    df[['RS14','RSI14']].tail()
    df.set_index('time', inplace = True)
    return(df)

def atualiza_historico(pasta='C:/Users/...'):
    from_date=datetime.now().strftime("%d-%m-%y")
    from_date=pd.to_datetime(from_date)
    to_date=datetime.now()
    historico=mt5.history_deals_get(from_date, to_date)
    df=pd.DataFrame(list(historico), columns=historico[0]._asdict().keys())
    df['time']=pd.to_datetime(df['time'], unit='s')
    df['Data'] = [d.date() for d in df['time']]
    df['Hora_Minuto'] = [d.time() for d in df['time']]
    df['DIRECAO']=" "
    df['DIRECAO'][df['type']==0]="COMPRA"
    df['DIRECAO'][df['type']==1]="VENDA"
    df['ESTADO']=" "
    df['ESTADO'][df['entry']==0]="ENTRADA"
    df['ESTADO'][df['entry']==1]="SAIDA"
    df=df[['Data','Hora_Minuto','symbol','DIRECAO','ESTADO','volume','price','profit','comment']]
    df.to_excel(pasta+'/historico-'+str(datetime.now().date())+'.xlsx', index = False)

def enviarEmail(remetente="...", senha="...", destinatario="...", tipo_operacao="...", ativo="WIN$", currentPrice=0, lucro=0, prejuizo=0, com_historico=0):
    import os
    import smtplib
    import imghdr
    from email.message import EmailMessage
    import datetime as dt
    import pandas as pd
    from pandas_datareader import data as pdr
    import time
    EMAIL_ADDRESS = remetente# digitar email que vai enviar
    #Precisa ir em Gerenciar sua Conta do Google->Segurança->Acesso a App menos Seguro-> Habilitar
    #Precisa ir em Gerenciar sua Conta do Google->Segurança->Como fazer login no Google->Verificação em duas etapas->Habilitar
    #Precisa ir em Gerenciar sua Conta do Google->Segurança->Como fazer login no Google->Senhas de app->Selecionar app=Email + Selecionar dispositivo = Computador Windows->Gerar
    #Copia a senha que aparecer e cola aqui em baixo
    EMAIL_PASSWORD = senha
    msg = EmailMessage()# <== that's all it takes :-)
    now = dt.datetime.now()
    stock=ativo
    #TargetPrice=180
    msg['Subject'] = 'Alert on '+ stock+'! '+tipo_operacao
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = destinatario
    #alerted=False
    #currentPrice=100
    #condition=currentClose>TargetPrice
    #if(condition and alerted==False):
    if tipo_operacao=="Entrada VENDA":
        message="Foi realizada uma operação de "+ tipo_operacao + " no ativo "+ stock +" no preço: "+ str(currentPrice)
    elif tipo_operacao=="Entrada COMPRA":
        message="Foi realizada uma operação de "+ tipo_operacao + " no ativo "+ stock +" no preço: "+ str(currentPrice)
    elif tipo_operacao=="Take Profit (vendido)":
        message="Foi realizada uma operação de "+ tipo_operacao + " no ativo "+ stock +" no preço: "+ str(currentPrice) +"\n com lucro de:"+ str(lucro)
    elif tipo_operacao=="Take Profit (comprado)":
        message="Foi realizada uma operação de "+ tipo_operacao + " no ativo "+ stock +" no preço: "+ str(currentPrice) +"\n com lucro de:"+ str(lucro)
    elif tipo_operacao=="Stop Loss (vendido)":
        message="Foi realizada uma operação de "+ tipo_operacao + " no ativo "+ stock +" no preço: "+ str(currentPrice) +"\n com prejuízo de:"+ str(prejuizo)
    elif tipo_operacao=="Stop Loss (comprado)":
        message="Foi realizada uma operação de "+ tipo_operacao + " no ativo "+ stock +" no preço: "+ str(currentPrice) +"\n com prejuízo de:"+ str(prejuizo)
    elif tipo_operacao=="teste":
        message="Hello World"
    elif tipo_operacao=="fechamento":
        message="HISTÓRICO APÓS FECHAMENTO DO DIA"
    #print(message)
    try:   
        msg.set_content(message)
        if(com_historico==1):
            atualiza_historico()
            files=['C:/Users/.../historico-'+str(datetime.now().date())+'.xlsx']
            for file in files:
                with open(file,'rb') as f:
                    file_data=f.read()
                    file_name='historico-'+str(datetime.now().date())+'.xlsx'
                    msg.add_attachment(file_data, maintype="application",
                        subtype='ocetet-stream', filename=file_name)
    except:
        print("An exception occurred")
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        smtp.send_message(msg)
        #print("completed")

def send_order(symbol, lot, price, desviation, otype, gain, stop, comment="...", hour_et="17:50", BuyORSell= mt5.ORDER_TYPE_BUY):
    mt5.symbol_select(symbol, True)
    if otype=="COMPRA":
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": float(lot),
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": price-stop,
            "tp": price+gain,
            "magic": 234000,
            "desviation": desviation,
            "comment": comment,
            "type_time":mt5.ORDER_TIME_GTC,
            'type_filling':mt5.ORDER_FILLING_RETURN
        }
        result = mt5.order_send(request)
    elif otype=="VENDA":
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": float(lot),
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": price+stop,
            "tp": price-gain,
            "magic": 234000,
            "desviation": desviation,
            "comment": comment,
            "type_time":mt5.ORDER_TIME_GTC,
            'type_filling':mt5.ORDER_FILLING_RETURN
        }
        result = mt5.order_send(request)
    elif otype=="VENDA PROTECAO":
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": float(lot),
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": price+stop,
            "tp": price-gain,
            "magic": 234000,
            "desviation": desviation,
            "comment": comment,
            "type_time":mt5.ORDER_TIME_GTC,
            'type_filling':mt5.ORDER_FILLING_RETURN
        }
        result = mt5.order_send(request)
    elif otype=="COMPRA PROTECAO":
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": float(lot),
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": price-stop,
            "tp": price+gain,
            "magic": 234000,
            "desviation": desviation,
            "comment": comment,
            "type_time":mt5.ORDER_TIME_GTC,
            'type_filling':mt5.ORDER_FILLING_RETURN
        }
        result = mt5.order_send(request)
    elif otype=="VENDA PARCIAL":
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": float(lot),
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": price+stop,
            "tp": price-gain,
            "magic": 234000,
            "desviation": desviation,
            "comment": comment,
            "type_time":mt5.ORDER_TIME_GTC,
            'type_filling':mt5.ORDER_FILLING_RETURN
        }
        result = mt5.order_send(request)
    elif otype=="COMPRA PARCIAL":
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": float(lot),
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": price-stop,
            "tp": price+gain,
            "magic": 234000,
            "desviation": desviation,
            "comment": comment,
            "type_time":mt5.ORDER_TIME_GTC,
            'type_filling':mt5.ORDER_FILLING_RETURN
        }
        result = mt5.order_send(request)
    elif otype=="EXPIRA":
        et=datetime.strptime(str(datetime.now().strftime("%d-%m-%Y")+" "+hour_et+":00"), "%d-%m-%Y %H:%M:%S")
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": float(lot),
            "type": mt5.ORDER_TYPE_SELL,
            "type_time": mt5.ORDER_TIME_SPECIFIED,
            "expiration": int(et.timestamp()),
            "price": price,
            "sl": price-stop,
            "tp": price+gain,
            "magic": 234000,
            "desviation": desviation,
            "comment": comment
            #"type_time":mt5.ORDER_TIME_GTC,
            #'type_filling':mt5.ORDER_FILLING_RETURN
        }
        print([et.timestamp(), et])
        result = mt5.order_send(request)
    return(result)

def retorna_volume():
    if(mt5.positions_total()>0):
        historico=mt5.positions_get()
        df=pd.DataFrame(list(historico), columns=historico[0]._asdict().keys())
        df['time']=pd.to_datetime(df['time'], unit='s')
        df['Data'] = [d.date() for d in df['time']]
        df['Hora_Minuto'] = [d.time() for d in df['time']]
        return(df['volume'][0])
    else:
        return(0)

def zera_posicoes(symbol="****"):
    if(mt5.positions_total()>0):
        historico=mt5.positions_get()
        df=pd.DataFrame(list(historico), columns=historico[0]._asdict().keys())
        df['time']=pd.to_datetime(df['time'], unit='s')
        df['Data'] = [d.date() for d in df['time']]
        df['Hora_Minuto'] = [d.time() for d in df['time']]
        df['volume'][0]
    else:
        return(0)

def inicializa_historico(pasta='C:/Users/**********/'):
    df=pd.DataFrame([1])
    df.to_excel(pasta+'/historico-'+str(datetime.now().date())+'.xlsx', index = False)
    #df.to_excel(pasta+'/historico-teste.xlsx', index = False)
def retorna_historico():
    try:
        from_date=datetime.now().date()
        from_date=pd.to_datetime(from_date)
        to_date=datetime.now()
        historico=mt5.history_deals_get(from_date, to_date)
        df=pd.DataFrame(list(historico), columns=historico[0]._asdict().keys())
        df['time']=pd.to_datetime(df['time'], unit='s')
        df['Data'] = [d.date() for d in df['time']]
        df['Hora_Minuto'] = [d.time() for d in df['time']]
        df['DIRECAO']=" "
        df['DIRECAO'][df['type']==0]="COMPRA"
        df['DIRECAO'][df['type']==1]="VENDA"
        df['ESTADO']=" "
        df['ESTADO'][df['entry']==0]="ENTRADA"
        df['ESTADO'][df['entry']==1]="SAIDA"
        df=df[['Data','Hora_Minuto','symbol','DIRECAO','ESTADO','volume','price','profit','comment']]
    except:
        df="Nenhuma operação realizada hoje"
    return(df)

In [ ]:
lgd="LIGADO"
time_frame="M5"
time_frame_mt5=mt5.TIMEFRAME_M5
mao=1
periodo_bb=20
desvio_bb=2.6
periodo_rsi=9
n_linhas=100
symbol='******' #ativo a ser operado
gain = 33
stop = 4
rsi_values=[22,78]
porcent_parcial=0.7
#bbw_values=[356,611]
t_inicio="10:00"
t_fim="14:30"
flag_interday=False
fez_parcial=False
i=1
email_remetente = "**********@gmail.com"
email_destinatario = "**********@gmail.com"
email_senha = "*****"
pasta="C:/Users/********"
inicializa_historico(pasta=pasta)
ope = 0
opd = 0 
acertos=0
while(1==1):
    df=get_ohlc(symbol, time_frame_mt5, n_linhas, periodo=periodo_bb, desvios=desvio_bb, window=periodo_rsi).tail(2)
    # BLOCO QUE REALIZA UMA OPERAÇÃO DE VENDA
    if ( ******condições para execução da ordem de venda********** & (df['Hora_Minuto'][1]>datetime.strptime(t_inicio, '%H:%M').time()) & (df['Hora_Minuto'][1]<datetime.strptime(t_fim, '%H:%M').time()) & (mt5.positions_total()==0):
        ope = mt5.symbol_info_tick(symbol).ask
        opd = -1
        #flagop = True
        #print("Abriu operação de VENDA---------------------------------------------------------")
        #print(df[['Datetime', 'open','high','low', 'close', 'Banda_Sup', 'Banda_Inf', 'RSI14']])
        send_order(symbol=symbol, lot=mao*2, price=mt5.symbol_info_tick(symbol).ask, desviation=2, otype="VENDA", 
                   gain=gain, stop=stop, comment="ENTRADA PRINCIPAL")
        enviarEmail(remetente=email_remetente, senha=email_senha, destinatario=email_destinatario,
                    tipo_operacao="Entrada VENDA", ativo=symbol, currentPrice=mt5.symbol_info_tick(symbol).ask, com_historico=1)
    # BLOCO QUE REALIZA UMA OPERAÇÃO DE COMPRA
    elif ( ******condições para execução da ordem de compra********** & (df['Hora_Minuto'][1]>datetime.strptime(t_inicio, '%H:%M').time()) & (df['Hora_Minuto'][1]<datetime.strptime(t_fim, '%H:%M').time()) & (mt5.positions_total()==0):
        ope = mt5.symbol_info_tick(symbol).bid
        opd = 1
        #flagop = True
        #print("Abriu operação de COMPRA---------------------------------------------------------")
        #print(df[['Datetime', 'open','high','low', 'close', 'Banda_Sup', 'Banda_Inf', 'RSI14']])
        send_order(symbol=symbol, lot=mao*2, price=mt5.symbol_info_tick(symbol).bid, desviation=2, otype="COMPRA", 
                   gain=gain, stop=stop, comment="ENTRADA PRINCIPAL")
        enviarEmail(remetente=email_remetente, senha=email_senha, destinatario=email_destinatario,
                    tipo_operacao="Entrada COMPRA", ativo=symbol, currentPrice=mt5.symbol_info_tick(symbol).bid, com_historico=1)
    #BLOCO QUE REALIZA O PARCIAL
    elif (mt5.positions_total()==1) & (retorna_volume()==mao*2) & (fez_parcial==False):
        if opd==-1:
            if df['low'][1]<=(ope-(gain*porcent_parcial)):
                send_order(symbol=symbol, lot=mao, price=mt5.symbol_info_tick(symbol).bid, desviation=2, otype="COMPRA PARCIAL",
                           comment="PARCIAL", gain=5, stop=(gain*porcent_parcial))
                #print("COMPRA PARCIAL")
                #print(df[['Datetime', 'open','high','low', 'close', 'Banda_Sup', 'Banda_Inf', 'RSI14']])
                #print([ope,ope-(gain*porcent_parcial)])
                fez_parcial=True
        if opd==1:
            if df['high'][1]>=(ope+(gain*porcent_parcial)):
                send_order(symbol=symbol, lot=mao, price=mt5.symbol_info_tick(symbol).ask, desviation=2, otype="VENDA PARCIAL", 
                           comment="PARCIAL",gain=5, stop=(gain*porcent_parcial))
                #print("VENDA PARCIAL")
                #print(df[['Datetime', 'open','high','low', 'close', 'Banda_Sup', 'Banda_Inf', 'RSI14']])
                #print([df['high'][1],ope,ope+(gain*porcent_parcial)])
                fez_parcial=True
    #BLOCO QUE POSICIONA A PROTEÇÃO QUANDO O PARCIAL É EXECUTADO
    elif (mt5.positions_total()==1) & (retorna_volume()==mao) & (fez_parcial==True):
        if opd==-1:
            if df['close'][1]>=(ope-10):
                send_order(symbol=symbol, lot=mao, price=mt5.symbol_info_tick(symbol).bid, desviation=2, otype="COMPRA PROTECAO", comment="COMPRA PROTECAO", gain=5, stop=5)
                #print("PROTEGEU")
                #print(df[['Datetime', 'open','high','low', 'close', 'Banda_Sup', 'Banda_Inf', 'RSI14']])
        if opd==1:
            if df['close'][1]<=(ope+10):
                send_order(symbol=symbol, lot=mao, price=mt5.symbol_info_tick(symbol).ask, desviation=2, otype="VENDA PROTECAO", comment="VENDA PROTECAO", gain=5, stop=5)
                #print("PROTEGEU")
                #print(df[['Datetime', 'open','high','low', 'close', 'Banda_Sup', 'Banda_Inf', 'RSI14']])
    elif (mt5.positions_total()==0):
        fez_parcial=False
    # BLOCO QUE IMPEDE QUE OPERAÇÃO CONTINUE PARA O PRÓXIMO DIA
    elif (df['Hora_Minuto'][1]>datetime.strptime('17:50', '%H:%M').time()) & (flag_interday==False) & (mt5.positions_total()==0):
        #flagop=False
        if (opd==-1) & (mt5.positions_total()>=1):
            send_order(symbol=symbol, lot=retorna_volume(), price=mt5.symbol_info_tick(symbol).bid, desviation=2, 
                       otype="COMPRA PROTECAO", comment="PROTECAO", gain=5, stop=5)
            #print("PROTEGEU interday=False")
            #print(df[['Datetime', 'open','high','low', 'close', 'Banda_Sup', 'Banda_Inf', 'RSI14']])
        if (opd==1) & (mt5.positions_total()>=1):
            send_order(symbol=symbol, lot=retorna_volume(), price=mt5.symbol_info_tick(symbol).ask, desviation=2, 
                       otype="VENDA PROTECAO", comment="PROTECAO", gain=5, stop=5)
            #print("PROTEGEU interday=False")
            #print(df[['Datetime', 'open','high','low', 'close', 'Banda_Sup', 'Banda_Inf', 'RSI14']])
    #print(msg)
    #if(df['Hora_Minuto'][i]>datetime.strptime(t_fim, '%H:%M').time()):
    #    enviarEmail(remetente=email_remetente, senha=email_senha, destinatario=email_destinatario,
    #              tipo_operacao="fechamento", ativo=symbol)
    df2=df[['close', 'Banda_Sup', 'Banda_Inf', 'RSI14']]
    clear_output(wait=True)
    if lgd=="...":
        lgd="LIGADO"
    else:
        lgd="..."
    display("ATIVO: "+symbol+" ------- VOLUME ABERTO:"+str(retorna_volume())+" ------- "+lgd)
    display("FEZ PARCIAL: "+str(fez_parcial))
    display("Periodo BB: "+str(periodo_bb)+" -- Desvio BB: "+str(desvio_bb)+" --- Periodo RSI: "+str(periodo_rsi))
    display("Gain: "+str(gain)+" --- Stop: "+str(stop)+" ---- limites RSI: "+str(rsi_values))
    display("Parcial em: "+ str(porcent_parcial*100)+"% ---- Inicio e fim: "+str([t_inicio, t_fim]))
    display("Permite interday: "+ str(flag_interday)+" --- TIMEFRAME: "+time_frame+" -- MÃO: "+ str(mao))
    #display(df2)
    display(retorna_historico())
    time.sleep(0.2)
    